In [4]:
# 0から順番に親要素の大きい方と入れ替える方法

from functools import lru_cache
import sys
sys.setrecursionlimit(10**9)
from collections import Counter
from itertools import permutations
from itertools import combinations
import math
import heapq
from collections import OrderedDict
import bisect
from collections import deque
from collections import defaultdict

def main():
    # key: (x,y)座標, value: その座標の値
    B_dict = defaultdict(int)
    # key: 値, value: (x,y)座標
    B_ind_dict = defaultdict(tuple)
    for i in range(30):
        B = list(map(int, input().split()))
        for j,b in enumerate(B):
            B_dict[(i,j)] = b
            B_ind_dict[b] = (i,j)
    # print(B_ind_dict)
    ans = []
    change_num = 0
    # ansへの追加は一回のループで一回だけ
    while len(ans) < 10000:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict[change_num]
        # print(change_num_ind)
        # change_numの親要素のうち値が大きい方のindexと値を取得
        parent_num = -1
        parent_ind = tuple()
        # 0段目以外で，左から0番目じゃなければ左上に要素がある
        if change_num_ind[0] > 0 and change_num_ind[1] > 0:
            if parent_num < B_dict[(change_num_ind[0]-1, change_num_ind[1]-1)]:
                parent_num = B_dict[(change_num_ind[0]-1, change_num_ind[1]-1)]
                parent_ind = (change_num_ind[0]-1, change_num_ind[1]-1)
        # 0段目以外で，x段目の左からx番目じゃなければ，右上に要素がある
        if change_num_ind[0] > 0 and change_num_ind[1] < change_num_ind[0]:
            if parent_num < B_dict[(change_num_ind[0]-1, change_num_ind[1])]:
                parent_num = B_dict[(change_num_ind[0]-1, change_num_ind[1])]
                parent_ind = (change_num_ind[0]-1, change_num_ind[1])
        # その座標の親座標の値より，change_numが小さければ
        if parent_num > change_num:
            # change_numの座標の右上，左上，の値の大きい方とchange_numの，値，座標を入れ替えて，ansに追加
            ans.append([change_num_ind[0], change_num_ind[1], parent_ind[0], parent_ind[1]])
            B_dict[change_num_ind], B_dict[parent_ind] = B_dict[parent_ind], B_dict[change_num_ind]
            B_ind_dict[change_num], B_ind_dict[parent_num] = B_ind_dict[parent_num], B_ind_dict[change_num]
        # その座標の親座標の値より，change_numが大きければ
        else:
            # change_numを+1して，次のループ
            change_num += 1
            if change_num == 465:
                break
    print(len(ans))
    for a in ans:
        print(*a)
    
if __name__ == "__main__":
    main()

defaultdict(<class 'tuple'>, {3: (0, 0), 1: (1, 0), 2: (1, 1), 5: (2, 0), 0: (2, 1), 4: (2, 2)})
(2, 1)
(1, 1)
(0, 0)
(1, 0)
(2, 1)
(1, 1)
(2, 1)
(2, 2)
(2, 0)
()


IndexError: tuple index out of range

In [10]:
# 0から順番に親要素の大きい方と入れ替える方法
# 0から順番に親要素の大きい方と入れ替えるとき，
# 親要素をBFSで遡っていって，最小手数で親より自分の方が大きくなるルートを選ぶ
# 多く見積もっても4495回の操作で終わるはずやから10000回の制限は気にしなくて大丈夫
from collections import deque
from collections import defaultdict

def main():
    N = 30
    # key: (x,y)座標, value: その座標の値
    B_dict = defaultdict(int)
    # key: 値, value: (x,y)座標
    B_ind_dict = defaultdict(tuple)
    for i in range(N):
        B = list(map(int, input().split()))
        for j,b in enumerate(B):
            B_dict[(i,j)] = b
            B_ind_dict[b] = (i,j)
    ans = []
    change_num = 0
    while True:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict[change_num]
        # 次に見るノードの座標が入ってるキュー
        q = deque([change_num_ind])
        # 後からルートを復元するために子要素を保存する辞書(key: 親要素のindex, value: 子要素のindex)
        parent_dict = defaultdict(tuple)
        # 最終的に行き着いたノード
        last_ind = tuple()
        # BFS(max_layer階層上までで打ち切る)
        # 何階層上まで見るか
        max_layer = 20
        # 打ち切ったかどうかを判定するフラグ
        flag = False
        max_ind = tuple()
        max_num = 0
        while q:
            # 今みてるノードの座標
            now_ind = q.popleft()
            # 親要素のうち値が大きい方のindexと値を取得
            parent_ind = tuple()
            # 0段目以外で，左から0番目じゃなければ左上に要素がある
            if now_ind[0] > 0 and now_ind[1] > 0:
                if change_num < B_dict[(now_ind[0]-1, now_ind[1]-1)]:
                    parent_ind = (now_ind[0]-1, now_ind[1]-1)
                    parent_dict[parent_ind] = now_ind
                    q.append((now_ind[0]-1, now_ind[1]-1))
                    if change_num_ind[0] - max_layer + 1 == now_ind[0]-1:
                        if max_num < B_dict[(now_ind[0]-1, now_ind[1]-1)]:
                            max_num = B_dict[(now_ind[0]-1, now_ind[1]-1)]
                            max_ind = (now_ind[0]-1, now_ind[1]-1)
                    if change_num_ind[0] - max_layer == now_ind[0]-1:
                        last_ind = max_ind
                        flag = True
                        break
            # 0段目以外で，x段目の左からx番目じゃなければ，右上に要素がある
            if now_ind[0] > 0 and now_ind[1] < now_ind[0]:
                if change_num < B_dict[(now_ind[0]-1, now_ind[1])]:
                    parent_ind = (now_ind[0]-1, now_ind[1])
                    parent_dict[parent_ind] = now_ind
                    q.append((now_ind[0]-1, now_ind[1]))
                    if change_num_ind[0] - max_layer + 1 == now_ind[0]-1:
                        if max_num < B_dict[(now_ind[0]-1, now_ind[1])]:
                            max_num = B_dict[(now_ind[0]-1, now_ind[1])]
                            max_ind = (now_ind[0]-1, now_ind[1])
                    if change_num_ind[0] - max_layer == now_ind[0]-1:
                        last_ind = max_ind
                        flag = True
                        break
            # 親要素がいないか，今見てるノードより値が小さいなら終了，このときの今見てるノードから遡っていく
            if parent_ind == tuple():
                last_ind = now_ind
                break
        # 遡っていってansに追加
        tmp = []
        while True:
            tmp.append(last_ind)
            if last_ind == change_num_ind:
                break
            last_ind = parent_dict[last_ind]
        # print(f"tmp: {tmp}")
        # 元いる場所がいい感じだった場合は何もせず，入れ替えがある時だけansに追加
        if len(tmp) >= 2:
            for i in range(1,len(tmp)):
                ans.append([tmp[-i][0], tmp[-i][1], tmp[-i-1][0], tmp[-i-1][1]])
                B_dict[tmp[-i]], B_dict[tmp[-i-1]] = B_dict[tmp[-i-1]], B_dict[tmp[-i]]
                B_ind_dict[B_dict[tmp[-i]]], B_ind_dict[B_dict[tmp[-i-1]]] = B_ind_dict[B_dict[tmp[-i-1]]], B_ind_dict[B_dict[tmp[-i]]]
        # change_numを+1して，次のループ
        # 打ち切った時は何もせずに次のループ
        if flag:
            continue
        change_num += 1
        if change_num == N*(N+1)//2:
            break
    print(len(ans))
    for a in ans:
        print(*a)
    
if __name__ == "__main__":
    main()

tmp: [(0, 0), (1, 1), (2, 2)]
tmp: [(1, 0), (2, 1)]
tmp: [(2, 0), (3, 0)]
tmp: [(1, 1), (2, 1)]
tmp: [(3, 0)]
tmp: [(2, 1), (3, 1)]
tmp: [(2, 2)]
tmp: [(3, 2)]
tmp: [(3, 3)]
tmp: [(3, 1)]
6
2 2 1 1
1 1 0 0
2 1 1 0
3 0 2 0
2 1 1 1
3 1 2 1


In [ ]:
# 0から順番に親要素の大きい方と入れ替える方法
# 0から順番に親要素の大きい方と入れ替えるとき，
# 親要素をBFSで遡っていって，最小手数で親より自分の方が大きくなるルートかつ，
# 出来るだけ，大きい数を選んでいくようなルートを選ぶ

In [ ]:
# 465から順番に子要素の値が小さい方と入れ替える方法
from collections import deque
from collections import defaultdict

def main():
    # key: (x,y)座標, value: その座標の値
    B_dict = defaultdict(int)
    # key: 値, value: (x,y)座標
    B_ind_dict = defaultdict(tuple)
    for i in range(30):
        B = list(map(int, input().split()))
        for j,b in enumerate(B):
            B_dict[(i,j)] = b
            B_ind_dict[b] = (i,j)
    ans = []
    change_num = 464
    # ansへの追加は一回のループで一回だけ
    while len(ans) < 10000:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict[change_num]
        # change_numの子要素のうち値が小さい方のindexと値を取得
        child_num = 465
        child_ind = tuple()
        # 30段目以外は左下と右下に要素がある
        if change_num_ind[0] < 29:
            # 左下，右下のうち小さい方をchile_num, child_indに入れる
            if B_dict[(change_num_ind[0]+1, change_num_ind[1])] < B_dict[(change_num_ind[0]+1, change_num_ind[1]+1)]:
                child_num = B_dict[(change_num_ind[0]+1, change_num_ind[1])]
                child_ind = (change_num_ind[0]+1, change_num_ind[1])
            else:
                child_num = B_dict[(change_num_ind[0]+1, change_num_ind[1]+1)]
                child_ind = (change_num_ind[0]+1, change_num_ind[1]+1)
        # その座標の子座標の値より，change_numが大きければ
        if child_num < change_num:
            # change_numの座標の左下，右下の値の小さい方とchange_numの，値，座標を入れ替えて，ansに追加
            ans.append([change_num_ind[0], change_num_ind[1], child_ind[0], child_ind[1]])
            B_dict[change_num_ind], B_dict[child_ind] = B_dict[child_ind], B_dict[change_num_ind]
            B_ind_dict[change_num], B_ind_dict[child_num] = B_ind_dict[child_num], B_ind_dict[change_num]
        # その座標の子座標の値より，change_numが小さければ
        else:
            # change_numを-1して，次のループ
            change_num -= 1
            if change_num == -1:
                break
    print(len(ans))
    for a in ans:
        print(*a)
    
if __name__ == "__main__":
    main()

In [ ]:
# 0から順番に親要素の大きい方と入れ替える方法と，
# 465から順番に子要素の値が小さい方と入れ替える方法を組み合わせて，手数が少ない方を選ぶ

from collections import deque
from collections import defaultdict

def main():
    # key: (x,y)座標, value: その座標の値
    B_dict_0 = defaultdict(int)
    B_dict_465 = defaultdict(int)
    # key: 値, value: (x,y)座標
    B_ind_dict_0 = defaultdict(tuple)
    B_ind_dict_465 = defaultdict(tuple)
    for i in range(30):
        B = list(map(int, input().split()))
        for j,b in enumerate(B):
            B_dict_0[(i,j)] = b
            B_dict_465[(i,j)] = b
            B_ind_dict_0[b] = (i,j)
            B_ind_dict_465[b] = (i,j)
    
    # 0から
    ans_0 = []
    change_num = 0
    # ansへの追加は一回のループで一回だけ
    while len(ans_0) < 10000:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict_0[change_num]
        # print(change_num_ind)
        # change_numの親要素のうち値が大きい方のindexと値を取得
        parent_num = -1
        parent_ind = tuple()
        # 0段目以外で，左から0番目じゃなければ左上に要素がある
        if change_num_ind[0] > 0 and change_num_ind[1] > 0:
            if parent_num < B_dict_0[(change_num_ind[0]-1, change_num_ind[1]-1)]:
                parent_num = B_dict_0[(change_num_ind[0]-1, change_num_ind[1]-1)]
                parent_ind = (change_num_ind[0]-1, change_num_ind[1]-1)
        # 0段目以外で，x段目の左からx番目じゃなければ，右上に要素がある
        if change_num_ind[0] > 0 and change_num_ind[1] < change_num_ind[0]:
            if parent_num < B_dict_0[(change_num_ind[0]-1, change_num_ind[1])]:
                parent_num = B_dict_0[(change_num_ind[0]-1, change_num_ind[1])]
                parent_ind = (change_num_ind[0]-1, change_num_ind[1])
        # その座標の親座標の値より，change_numが小さければ
        if parent_num > change_num:
            # change_numの座標の右上，左上，の値の大きい方とchange_numの，値，座標を入れ替えて，ansに追加
            ans_0.append([change_num_ind[0], change_num_ind[1], parent_ind[0], parent_ind[1]])
            B_dict_0[change_num_ind], B_dict_0[parent_ind] = B_dict_0[parent_ind], B_dict_0[change_num_ind]
            B_ind_dict_0[change_num], B_ind_dict_0[parent_num] = B_ind_dict_0[parent_num], B_ind_dict_0[change_num]
        # その座標の親座標の値より，change_numが大きければ
        else:
            # change_numを+1して，次のループ
            change_num += 1
            if change_num == 465:
                break

    # 465から
    ans_465 = []
    change_num = 464
    # ansへの追加は一回のループで一回だけ
    while len(ans_465) < 10000:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict_465[change_num]
        # change_numの子要素のうち値が小さい方のindexと値を取得
        child_num = 465
        child_ind = tuple()
        # 30段目以外は左下と右下に要素がある
        if change_num_ind[0] < 29:
            # 左下，右下のうち小さい方をchile_num, child_indに入れる
            if B_dict_465[(change_num_ind[0]+1, change_num_ind[1])] < B_dict_465[(change_num_ind[0]+1, change_num_ind[1]+1)]:
                child_num = B_dict_465[(change_num_ind[0]+1, change_num_ind[1])]
                child_ind = (change_num_ind[0]+1, change_num_ind[1])
            else:
                child_num = B_dict_465[(change_num_ind[0]+1, change_num_ind[1]+1)]
                child_ind = (change_num_ind[0]+1, change_num_ind[1]+1)
        # その座標の子座標の値より，change_numが大きければ
        if child_num < change_num:
            # change_numの座標の左下，右下の値の小さい方とchange_numの，値，座標を入れ替えて，ansに追加
            ans_465.append([change_num_ind[0], change_num_ind[1], child_ind[0], child_ind[1]])
            B_dict_465[change_num_ind], B_dict_465[child_ind] = B_dict_465[child_ind], B_dict_465[change_num_ind]
            B_ind_dict_465[change_num], B_ind_dict_465[child_num] = B_ind_dict_465[child_num], B_ind_dict_465[change_num]
        # その座標の子座標の値より，change_numが小さければ
        else:
            # change_numを-1して，次のループ
            change_num -= 1
            if change_num == -1:
                break
    
    if len(ans_0) < len(ans_465):
        ans = ans_0
    else:
        ans = ans_465
    print(len(ans))
    for a in ans:
        print(*a)
    
if __name__ == "__main__":
    main()

In [ ]:
# 0から順番に親要素の大きい方と入れ替える方法
# 0から順番に親要素の大きい方と入れ替えるとき，
# 親要素をBFSで遡っていって，最小手数で親より自分の方が大きくなるルートを選ぶ
# 多く見積もっても4495回の操作で終わるはずやから10000回の制限は気にしなくて大丈夫
from collections import deque
from collections import defaultdict

def childs_ind(now_ind, change_ind):
    tmp = []
    # 30段目以外は左下と右下に要素がある
    if now_ind[0] < 29:
        # now_ind[1]+1がchange_ind[1]以下ならtmpに追加
        if now_ind[1]+1 <= change_ind[1]:
            tmp.append((now_ind[0]+1, now_ind[1]+1))
        # (change_ind[0]-(now_ind[0]+1)) - (change_ind[1]-now_ind[1]) >= 0ならtmpに追加
        if (change_ind[0]-now_ind[0]-1) - (change_ind[1]-now_ind[1]) >= 0:
            tmp.append((now_ind[0]+1, now_ind[1]))
    return tmp

def main():
    N = 30
    # key: (x,y)座標, value: その座標の値
    B_dict = defaultdict(int)
    # key: 値, value: (x,y)座標
    B_ind_dict = defaultdict(tuple)
    for i in range(N):
        B = list(map(int, input().split()))
        for j,b in enumerate(B):
            B_dict[(i,j)] = b
            B_ind_dict[b] = (i,j)
    ans = []
    change_num = 0
    while True:
        # 値がchange_numの座標を見て，
        change_num_ind = B_ind_dict[change_num]
        # 次に見るノードの座標が入ってるキュー
        q = deque([change_num_ind])
        # 最終的に行き着いたノード
        last_ind = tuple()
        # BFS(max_layer階層上までで打ち切る)
        # 何階層上まで見るか
        max_layer = 23
        # 打ち切ったかどうかを判定するフラグ
        flag = False
        max_ind = tuple()
        max_num = 0
        while q:
            # 今みてるノードの座標
            now_ind = q.popleft()
            # 親要素のうち値が大きい方のindexと値を取得
            parent_ind = tuple()
            # 0段目以外で，左から0番目じゃなければ左上に要素がある
            if now_ind[0] > 0 and now_ind[1] > 0:
                if change_num < B_dict[(now_ind[0]-1, now_ind[1]-1)]:
                    parent_ind = (now_ind[0]-1, now_ind[1]-1)
                    q.append((now_ind[0]-1, now_ind[1]-1))
                    if change_num_ind[0] - max_layer + 1 == now_ind[0]-1:
                        if max_num < B_dict[(now_ind[0]-1, now_ind[1]-1)]:
                            max_num = B_dict[(now_ind[0]-1, now_ind[1]-1)]
                            max_ind = (now_ind[0]-1, now_ind[1]-1)
                    if change_num_ind[0] - max_layer == now_ind[0]-1:
                        last_ind = max_ind
                        flag = True
                        break
            # 0段目以外で，x段目の左からx番目じゃなければ，右上に要素がある
            if now_ind[0] > 0 and now_ind[1] < now_ind[0]:
                if change_num < B_dict[(now_ind[0]-1, now_ind[1])]:
                    parent_ind = (now_ind[0]-1, now_ind[1])
                    q.append((now_ind[0]-1, now_ind[1]))
                    if change_num_ind[0] - max_layer + 1 == now_ind[0]-1:
                        if max_num < B_dict[(now_ind[0]-1, now_ind[1])]:
                            max_num = B_dict[(now_ind[0]-1, now_ind[1])]
                            max_ind = (now_ind[0]-1, now_ind[1])
                    if change_num_ind[0] - max_layer == now_ind[0]-1:
                        last_ind = max_ind
                        flag = True
                        break
            # 親要素がいないか，今見てるノードより値が小さいなら終了，このときの今見てるノードから遡っていく
            if parent_ind == tuple():
                last_ind = now_ind
                break
        # 遡っていってansに追加
        tmp = []
        # print()
        while True:
            tmp.append(last_ind)
            # print(tmp)
            if last_ind == change_num_ind:
                break
            nexts_ind = childs_ind(last_ind, change_num_ind)
            if len(nexts_ind) == 1:
                last_ind = nexts_ind[0]
            else:
                # print(last_ind, change_num_ind, nexts_ind)
                if B_dict[nexts_ind[0]] > B_dict[nexts_ind[1]]:
                    # print("a")
                    last_ind = nexts_ind[0]
                else:
                    # print("b")
                    last_ind = nexts_ind[1]
        # print(f"tmp: {tmp}")
        # 元いる場所がいい感じだった場合は何もせず，入れ替えがある時だけansに追加
        if len(tmp) >= 2:
            for i in range(1,len(tmp)):
                ans.append([tmp[-i][0], tmp[-i][1], tmp[-i-1][0], tmp[-i-1][1]])
                B_dict[tmp[-i]], B_dict[tmp[-i-1]] = B_dict[tmp[-i-1]], B_dict[tmp[-i]]
                B_ind_dict[B_dict[tmp[-i]]], B_ind_dict[B_dict[tmp[-i-1]]] = B_ind_dict[B_dict[tmp[-i-1]]], B_ind_dict[B_dict[tmp[-i]]]
        # change_numを+1して，次のループ
        # 打ち切った時は何もせずに次のループ
        if flag:
            continue
        change_num += 1
        if change_num == N*(N+1)//2:
            break
    print(len(ans))
    for a in ans:
        print(*a)
    
if __name__ == "__main__":
    main()